# Importing packages

In [1]:
import requests
#from keras.applications import ResNet50
#from keras.applications.resnet import decode_predictions, preprocess_input 
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
from dotenv import load_dotenv
import pandas as pd
from sklearn.model_selection import train_test_split
from helpers import *

# Load environment variables from the secret.env file
load_dotenv()

# Access environment variables using os.getenv() or os.environ
api_key = os.getenv("API_KEY")
api_url = os.getenv("API_URL")

2024-12-20 16:14:52.717763: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 16:14:52.718146: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 16:14:52.720195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 16:14:52.726129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734707692.738059   10890 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734707692.74

In [2]:
available_models = [name for name in dir(tf.keras.applications) if 'resnet' in name.lower()]
print(available_models)

['InceptionResNetV2', 'ResNet101', 'ResNet101V2', 'ResNet152', 'ResNet152V2', 'ResNet50', 'ResNet50V2', 'inception_resnet_v2', 'resnet', 'resnet50', 'resnet_v2']


In [3]:
# Example usage
#model_name = "ResNet50"  # Can be ResNet101, ResNet50V2, etc.
model_name = "ResNet101"
input_shape = (224, 224, 3)

base_model = load_resnet_basemodel(model_name, input_shape=input_shape)

# Display the model summary
# model.summary()

Model 'ResNet101' found in tf.keras.applications.


2024-12-20 16:14:54.810900: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Base_model 'ResNet101' loaded successfully.


# Get images through API call

In [4]:
tag = "vine"
start_date = "2020-08-01"
end_date = "2021-01-01"
tags = ['grass']

In [5]:
'''tag = "vine"
start_date = "2014-01-01"
end_date = "2025-01-01"'''

'tag = "vine"\nstart_date = "2014-01-01"\nend_date = "2025-01-01"'

In [6]:
tags = ['vine']
#tags = ['vine', 'grass', 'ground']

In [7]:
image_urls = get_image_urls_with_multiple_tags(tags, start_date, end_date, api_key, api_url)

Number of urls collected for vine: 23


# Download image for urls and make the training and test datasets

In [8]:
df_sample_map = download_images_and_create_sample_map(image_urls)

Image Signature=55709ae943dabb1c89e8be508f697c7eb8342d0f downloaded successfully
Image Signature=fbaeea5561997a82eb6cb99b1f9ba57d1325f0c9 downloaded successfully
Image Signature=57abdd6461226256b56d990610eda14eca8626dc downloaded successfully
Image Signature=4b7440f33e4ad8c49f1338297325f25abe01e43e downloaded successfully
Image Signature=884c68fbcf4d08c44aba91f8829d08cefe968369 downloaded successfully
Image Signature=89675e227e2db65e862fd95a662025137aec2948 downloaded successfully
Image Signature=1e78767e068341b78940a1c28d0a4af7141f08f6 downloaded successfully
Image Signature=7c1eca2cf95b6937e9a99e6ffe8deea98dcfa266 downloaded successfully
Image Signature=25edfd8287cfaa82f1a06ca48344b0ddd2428360 downloaded successfully
Image Signature=148182e3be8c7cc5e51cf4cd90be18ce890d7e83 downloaded successfully
Image Signature=3f162a4fef6bde2bd4492d4f8c27438145f66582 downloaded successfully
Image Signature=2507355b43bee932ba58d564c6354c431e226f8f downloaded successfully
Image Signature=32c3d385be92

In [9]:
image_dir = "media"
image_paths, encoded_labels = create_path_list_and_encoded_label(df_sample_map, image_dir) 

In [10]:
#image_paths

In [11]:
#encoded_labels

In [12]:
batch_size = 5
random_state = 42
test_size = 0.2
train_dataset, val_dataset = make_train_test_dataset(image_paths, encoded_labels, test_size=test_size, random_state=random_state, batch_size=batch_size, model_name=model_name)

Preprocess_input function for 'ResNet101' loaded successfully.


In [ ]:
# Load ResNet50 model pre-trained on ImageNet
#base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Allowing the base model to be trained for better performance
base_model.trainable = True

# Add custom layers on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 1
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

4/4 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.5222 - loss: 0.9655 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [14]:
stop

NameError: name 'stop' is not defined

# Train the model

In [51]:
# Load ResNet50 model pre-trained on ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Allowing the base model to be trained for better performance
base_model.trainable = True

# Add custom layers on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 1
history = model.fit(dataset, epochs=EPOCHS)


ValueError: Input 0 of layer "functional_3" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(224, 224, 3)

# Get timestamp

In [ ]:
image_vine_timestamp = [i['timestamp'].split('T')[0] for i in image_urls_vines]
image_ground_timestamp = [i['timestamp'].split('T')[0] for i in image_urls_vines]
image_grass_timestamp = [i['timestamp'].split('T')[0] for i in image_urls_vines]

image_grass_timestamp[:10]

import pandas as pd

# Create a DataFrame with the timestamps
df_timestamps_vine = pd.DataFrame({'date': image_vine_timestamp})
df_timestamps_vine['type'] = 'vine'                            
df_timestamps_grass = pd.DataFrame({'date': image_grass_timestamp})
df_timestamps_grass['type'] = 'grass'
df_timestamps_ground = pd.DataFrame({'date': image_ground_timestamp})
df_timestamps_ground['type'] = 'ground'
df_timestamps = pd.concat([df_timestamps_vine, df_timestamps_grass, df_timestamps_ground])

# Convert the timestamps to datetime objects
df_timestamps['date'] = pd.to_datetime(df_timestamps['date'])

# Group by Date and count unique classes
unique_type_per_day = df_timestamps.groupby(['date', 'type'], as_index=False).agg(lambda x: x.shape[0])

unique_type_per_day.columns = ['date', 'type', 'count']
unique_type_per_day.plot(x='date', y='count', kind='bar', figsize=(15, 6))